# Requirements Generator and MoSCoW Prioritization
**Author:** Gael Sánchez
**LinkedIn:** www.linkedin.com/in/gaelsanchez

This notebook generates and validates functional and non-functional software requirements from a natural language description, and classifies them using the MoSCoW prioritization technique.



## What is a MoSCoW Matrix?

The MoSCoW Matrix is a prioritization technique used in software development to categorize requirements based on their importance and urgency. The acronym stands for:

- **Must Have** – Critical requirements that are essential for the system to function.  
- **Should Have** – Important requirements that add significant value, but are not critical for initial delivery.  
- **Could Have** – Nice-to-have features that can enhance the product, but are not necessary.  
- **Won’t Have (for now)** – Low-priority features that will not be implemented in the current scope.

This method helps development teams make clear decisions about what to focus on, especially when working with limited time or resources. It ensures that the most valuable and necessary features are delivered first, contributing to better project planning and stakeholder alignment.


## How it works

This notebook uses the OpenAI library (via the Gemini API) to extract and validate software requirements from a natural language description. The workflow follows these steps:

1. **Initial Validation**  
   The user provides a textual description of the software. The model evaluates whether the description contains enough information to derive meaningful requirements. Specifically, it checks if the description answers key questions such as:
   
   - What is the purpose of the software?  
   - Who are the intended users?  
   - What are the main features and functionalities?  
   - What platform(s) will it run on?  
   - How will data be stored or persisted?  
   - Is authentication/authorization needed?  
   - What technologies or frameworks will be used?  
   - What are the performance expectations?  
   - Are there UI/UX principles to follow?  
   - Are there external integrations or dependencies?  
   - Will it support offline usage?  
   - Are advanced features planned?  
   - Are there security or privacy concerns?  
   - Are there any constraints or limitations?  
   - What is the timeline or development roadmap?

   If the description lacks important details, the model requests the missing information from the user. This loop continues until the model considers the description complete.

2. **Summarization**  
   Once validated, the model summarizes the software description, extracting its key aspects to form a concise and informative overview.

3. **Requirements Generation**  
   Using the summary, the model generates a list of functional and non-functional requirements.

4. **Requirements Validation**  
   A separate validation step checks if the generated requirements are complete and accurate based on the summary. If not, the model provides feedback, and the requirements are regenerated accordingly. This cycle repeats until the validation step approves the list.

5. **MoSCoW Prioritization**  
   Finally, the validated list of requirements is classified using the MoSCoW prioritization technique, grouping them into:
   
   - Must have  
   - Should have  
   - Could have  
   - Won't have for now

The output is a clear, structured requirements matrix ready for use in software development planning.


## Example Usage

### Input

**Software Name:** Personal Task Manager  
**Initial Description:**  
This will be a simple desktop application that allows users to create, edit, mark as completed, and delete daily tasks. Each task will have a title, an optional description, a due date, and a status (pending or completed). The goal is to help users organize their activities efficiently, with an intuitive and minimalist interface.

**Main Features:**

- Add new tasks  
- Edit existing tasks  
- Mark tasks as completed  
- Delete tasks  
- Filter tasks by status or date

**Additional Context Provided After Model Request:**

- **Intended Users:** Individuals seeking to improve their daily productivity, such as students, remote workers, and freelancers.  
- **Platform:** Desktop application for common operating systems.  
- **Data Storage:** Tasks will be stored locally.  
- **Authentication/Authorization:** A lightweight authentication layer may be included for data protection.  
- **Technology Stack:** Cross-platform technologies that support a modern, functional UI.  
- **Performance:** Expected to run smoothly with a reasonable number of active and completed tasks.  
- **UI/UX:** Prioritizes a simple, modern user experience.  
- **Integrations:** Future integration with calendar services is considered.  
- **Offline Usage:** The application will work without an internet connection.  
- **Advanced Features:** Additional features like notifications or recurring tasks may be added in future versions.  
- **Security/Privacy:** User data privacy will be respected and protected.  
- **Constraints:** Focus on simplicity, excluding complex features in the initial version.  
- **Timeline:** Development planned in phases, starting with a functional MVP.

### Output

**MoSCoW Prioritization Matrix:**

**Must Have**
- Task Creation: [The system needs to allow users to add tasks to be functional.]  
- Task Editing: [Users must be able to edit tasks to correct mistakes or update information.]  
- Task Completion: [Marking tasks as complete is a core function of a task management system.]  
- Task Deletion: [Users need to be able to remove tasks that are no longer relevant.]  
- Task Status: [Maintaining task status (pending/completed) is essential for tracking progress.]  
- Data Persistence: [Tasks must be stored to be useful beyond a single session.]  
- Performance: [The system needs to perform acceptably for a reasonable number of tasks.]  
- Usability: [The system must be easy to use for all other functionalities to be useful.]

**Should Have**
- Task Filtering by Status: [Filtering enhances usability and allows users to focus on specific tasks.]  
- Task Filtering by Date: [Filtering by date helps manage deadlines.]  
- User Interface Design: [A modern design improves user experience.]  
- Platform Compatibility: [Running on common OSes increases adoption.]  
- Data Privacy: [Important for user trust, can be gradually improved.]  
- Security: [Basic protections are necessary, advanced features can wait.]

**Could Have**
- Optional Authentication: [Enhances security but adds complexity.]  
- Offline Functionality: [Convenient, but not critical for MVP.]

**Won’t Have (for now)**
- N/A: [No features were excluded completely at this stage.]

---

This example demonstrates how the notebook takes a simple description and iteratively builds a complete and validated set of software requirements, ultimately organizing them into a MoSCoW matrix for development planning.


In [14]:
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel
import gradio as gr

In [ ]:
load_dotenv(override=True)


In [16]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
    


In [17]:
class StandardSchema(BaseModel):
    understood: bool
    feedback: str
    output: str

In [18]:
# This is the prompt to validate the description of the software product on the first step
system_prompt = f"""
        You are a software analyst. the user will give you a description of a software product. Your task is to decide the description provided is complete and accurate and useful to derive requirements for the software.
        If you decide the description is not complete or accurate, you should provide a kind message to the user listing the missing or incorrect information, and ask them to provide the missing information.
        If you decide the description is complete and accurate, you should provide a summary of the description in a structured format. Only provide the summary, nothing else.
        Ensure that the description answers the following questions:
        - What is the purpose of the software?
        - Who are the intended users?
        - What are the main features and functionalities of the software?
        - What platform(s) will it run on?
        - How will data be stored or persisted?
        - Is user authentication or authorization required?
        - What technologies or frameworks will be used?
        - What are the performance expectations?
        - Are there any UI/UX design principles that should be followed?
        - Are there any external integrations or dependencies?
        - Will it support offline usage?
        - Are there any planned advanced features?
        - Are there any security or privacy considerations?
        - Are there any constrains or limitations?
        - What is the desired timeline or development roadmap?

        Respond in the following format:
        
        "understood": true only if the description is complete and accurate
        "feedback": Instructions to the user to provide the missing or incorrect information.
        "output": Summary of the description in a structured format, once the description is complete and accurate.
        
    """

In [19]:
# This function is used to validate the description and provide feedback to the user.
# It receives the messages from the user and the system prompt.
# It returns the validation response.

def validate_and_feedback(messages):

    validation_response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=StandardSchema)
    validation_response = validation_response.choices[0].message.parsed
    return validation_response


In [20]:
# This function is used to validate the requirements and provide feedback to the model.
# It receives the description and the requirements.
# It returns the validation response.

def validate_requirements(description, requirements):
    validator_prompt = f"""
        You are a software requirements reviewer.
        Your task is to analyze a set of functional and non-functional requirements based on a given software description.

        Perform the following validation steps:

        Completeness: Check if all key features, fields, and goals mentioned in the description are captured as requirements.

        Consistency: Verify that all listed requirements are directly supported by the description. Flag anything that was added without justification.

        Clarity & Redundancy: Identify requirements that are vague, unclear, or redundant.

        Missing Elements: Highlight important elements from the description that were not translated into requirements.

        Suggestions: Recommend improvements or additional requirements that better align with the description.

        Answer in the following format:
        
        "understood": true only if the requirements are complete and accurate,
        "feedback": Instructions to the generator to improve the requirements.
        
        Here's the software description:
        {description}

        Here's the requirements:
        {requirements}

    """

    validator_response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=[{"role": "user", "content": validator_prompt}], response_format=StandardSchema)
    validator_response = validator_response.choices[0].message.parsed
    return validator_response


In [21]:
# This function is used to generate a rerun prompt for the requirements generator.
# It receives the description, the requirements and the feedback.
# It returns the rerun prompt.

def generate_rerun_requirements_prompt(description, requirements, feedback):
    return f"""
    You are a software analyst. Based on the following software description, you generated the following list of functional and non-functional requirements. 
    However, the requirements validator rejected the list, with the following feedback. Please review the feedback and improve the list of requirements.

    ## Here's the description:
    {description}

    ## Here's the requirements:
    {requirements}

    ## Here's the feedback:
    {feedback}
    """

In [22]:
# This function generates the requirements based on the description.
def generate_requirements(description):
    generator_prompt = f"""
    You are a software analyst. Based on the following software description, generate a comprehensive list of both functional and non-functional requirements.

    The requirements must be clear, actionable, and written in concise natural language.

    Each requirement should describe exactly what the system must do or how it should behave, with enough detail to support MoSCoW prioritization and later transformation into user stories.

    Group the requirements into two sections: Functional Requirements and Non-Functional Requirements.

    Avoid redundancy. Do not include implementation details unless they are part of the expected behavior.

    Write in professional and neutral English.

    Output in Markdown format.

    Answer in the following format:

    "understood": true
    "output": List of requirements

    ## Here's the description:
    {description}

    ## Requirements:
    """

    requirements_response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=[{"role": "user", "content": generator_prompt}], response_format=StandardSchema)
    requirements_response = requirements_response.choices[0].message.parsed
    requirements = requirements_response.output

    requirements_valid = validate_requirements(description, requirements)
    
    # Validation loop
    while not requirements_valid.understood:
        rerun_requirements_prompt = generate_rerun_requirements_prompt(description, requirements, requirements_valid.feedback)
        requirements_response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=[{"role": "user", "content": rerun_requirements_prompt}], response_format=StandardSchema)
        requirements_response = requirements_response.choices[0].message.parsed
        requirements = requirements_response.output
        requirements_valid = validate_requirements(description, requirements)

    return requirements



In [23]:
# This function generates the MoSCoW priorization of the requirements.
# It receives the requirements.
# It returns the MoSCoW priorization.

def generate_moscow_priorization(requirements):
    priorization_prompt = f"""
    You are a product analyst.
    Based on the following list of functional and non-functional requirements, classify each requirement into one of the following MoSCoW categories:

    Must Have: Essential requirements that the system cannot function without.

    Should Have: Important requirements that add significant value but are not absolutely critical.

    Could Have: Desirable but non-essential features, often considered nice-to-have.

    Won’t Have (for now): Requirements that are out of scope for the current version but may be included in the future.

    For each requirement, place it under the appropriate category and include a brief justification (1–2 sentences) explaining your reasoning.

    Format your output using Markdown, like this:

    ## Must Have
    - [Requirement]: [Justification]

    ## Should Have
    - [Requirement]: [Justification]

    ## Could Have
    - [Requirement]: [Justification]

    ## Won’t Have (for now)
    - [Requirement]: [Justification]

    ## Here's the requirements:
    {requirements}
    """

    priorization_response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=[{"role": "user", "content": priorization_prompt}], response_format=StandardSchema)
    priorization_response = priorization_response.choices[0].message.parsed
    priorization = priorization_response.output
    return priorization





In [24]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    validation =validate_and_feedback(messages)

    if not validation.understood:
        print('retornando el feedback')
        return validation.feedback
    else:
        requirements = generate_requirements(validation.output)
        moscow_prioritization = generate_moscow_priorization(requirements)
        return moscow_prioritization
        

In [ ]:
gr.ChatInterface(chat, type="messages").launch()